In [62]:
from easyquant import MongoIo
from easyquant.indicator.base import *
import math
import numpy as np
import pandas as pd
from easyquant import MongoIo
from easyquant.qafetch import QATdx as tdx
from easyquant.indicator.base import *
import pandas as pd
from easyquant import EasyTime
from datetime import datetime, time
# import time

In [63]:
et=EasyTime()

In [65]:
datetime.now()

datetime.datetime(2020, 6, 29, 15, 3, 31, 89419)

In [66]:
et.get_next_trade_date(datetime.now())

datetime.datetime(2020, 6, 30, 15, 3, 35, 572963)

In [ ]:
m=MongoIo()

In [ ]:
data=m.get_stock_day('600822',st_start='2020-02-01', st_end='2020-12-07')

In [ ]:
C=data.close
H=data.high
L=data.low